In [20]:
import pandas as pd
import requests, json, os, folium
from urllib.parse import quote
import numpy as np

In [21]:
df = pd.read_csv('data/서울부동산원본.csv')
df

,BJDONG_NM,ADDRESS,RA_REGNO,RDEALER_NM,CMP_NM,TELNO
0,신도림동,"서울특별시 구로구 경인로 661 101동 619호(신도림동, 신도림1차푸르지오)",11500-2022-00106,이동근,라빌공인중개사사무소,NaN
1,구로동,"서울특별시 구로구 디지털로33길 11 101-1호(구로동, 에이스테크노타워8차)",11530-2023-00039,이지흔,타워공인중개사사무소,02-2025-7677
2,구로동,서울특별시 구로구 구로동로 117 1층 일부 (구로동),41190-2022-00036,이창우,아라공인중개사사무소,NaN
3,구로동,서울특별시 구로구 도림로3길 3 1층 일부(구로동),11530-2023-00041,조영매,한중공인중개사사무소,NaN
4,구로동,"서울특별시 구로구 새말로 31 롯데아파트상가 108호(구로동, 신도림롯데아파트)",92420000-977,최발,롯데공인중개사사무소,855-8000
...,...,...,...,...,...,...
26603,서교동,"서울특별시 마포구 월드컵로10길 13 , 1층우측 (서교동)",11440-2015-00119,최지현,리치공인중개사사무소,-
26604,신수동,서울특별시 마포구 독막로 240 (신수동),92380000-4891,정숙영,성원부동산공인중개사사무소,715-7989
26605,노고산동,"서울특별시 마포구 신촌로 150 102호 (노고산동, 신촌포스빌)",92380000-4894,이옥현,포스빌부동산공인중개사사무소,702-2229
26606,공덕동,서울특별시 마포구 마포대로7길 33 (공덕동),92380000-4906,이병규,굿모닝부동산공인중개사사무소,711-0509


In [22]:
df_sorted = df.sort_values(by='ADDRESS', key=lambda x: x.str.encode('utf-8').str.decode('utf-8'))
df_sorted

,BJDONG_NM,ADDRESS,RA_REGNO,RDEALER_NM,CMP_NM,TELNO
7178,신사동,서울특별시 강남구 가로수길 16 1층,나-9250-46,이종학,우림부동산중개사무소,544-9659
5795,신사동,서울특별시 강남구 가로수길 27 6층[603호](신사동),11680-2023-00008,이원미,가로재공인중개사사무소,-
7191,신사동,서울특별시 강남구 가로수길 59 1층(신사동),11680-2020-00512,황창용,강남부동산중개,-
21235,신사동,서울특별시 강남구 가로수길 60 2층(신사동),가-9250-2404,심순보,동방부동산중개사무소,542-8818
14161,도곡동,서울특별시 강남구 강남대로 240 B105호(도곡동),11680-2016-00476,홍인호,SK허브공인중개사사무소,576-0618
...,...,...,...,...,...,...
22192,상봉동,"서울특별시 중랑구 중랑천로12길 42 101호(상봉동, 연다울빌)",11260-2021-00064,송영일,랜드고베스트부동산중개법인,02-433-3366\n02-2039-1184\n02-2039-1185\n02-20...
264,중화동,"서울특별시 중랑구 중랑천로14길 48 101호(더샤이닝, 중화동)",11260-2020-00011,박소은,더샤이닝공인중개사사무소,02-2207-8500
26379,중화동,서울특별시 중랑구 중랑천로14길 58 범양프레체1층3호(중화동),11260-2019-00073,김대경,중랑역범양공인중개사사무소,02-433-4343
23125,중화동,서울특별시 중랑구 중랑천로20길 20 (중화동),11260-2015-00096,김성호,여명공인중개사사무소,2249-7322


In [23]:
df.drop(columns=['RDEALER_NM', 'TELNO'], inplace=True)
df.head(20)

,BJDONG_NM,ADDRESS,RA_REGNO,CMP_NM
0,신도림동,"서울특별시 구로구 경인로 661 101동 619호(신도림동, 신도림1차푸르지오)",11500-2022-00106,라빌공인중개사사무소
1,구로동,"서울특별시 구로구 디지털로33길 11 101-1호(구로동, 에이스테크노타워8차)",11530-2023-00039,타워공인중개사사무소
2,구로동,서울특별시 구로구 구로동로 117 1층 일부 (구로동),41190-2022-00036,아라공인중개사사무소
3,구로동,서울특별시 구로구 도림로3길 3 1층 일부(구로동),11530-2023-00041,한중공인중개사사무소
4,구로동,"서울특별시 구로구 새말로 31 롯데아파트상가 108호(구로동, 신도림롯데아파트)",92420000-977,롯데공인중개사사무소
5,가리봉동,서울특별시 구로구 디지털로20길 13 1층일부(가리봉동),92420000-2608,부자공인중개사사무소
6,구로동,"서울특별시 구로구 공원로6길 25 상가105호(구로동, 비즈트위트레인보우)",11530-2022-00029,21세기공인중개사사무소
7,구로동,"서울특별시 구로구 디지털로29길 38 109호 (구로동, 에이스테크노타워Ⅲ)",11545-2017-00022,현대부동산공인중개사사무소
8,신도림동,서울특별시 구로구 경인로 621,92420000-1065,광장공인중개사사무소
9,신도림동,서울특별시 구로구 신도림로 33-1 (신도림동),92420000-1413,시온공인중개사사무소
